# Deployment Monitoring Integration

## 0. Imports

In [ ]:
from __future__ import print_function

import itertools
import os
import time

import six

import numpy as np
import pandas as pd

import sklearn
from sklearn import linear_model

### 0.1 Verta import and setup

In [ ]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

In [ ]:
import os

# Ensure credentials are set up, if not, use below
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST'] = 

from verta import Client
client = Client(os.environ['VERTA_HOST'])

## 1. Model Training

### 1.1 Load training data

In [ ]:
try:
    import wget
except ImportError:
    !pip install wget  # you may need pip3
    import wget

In [ ]:
train_data_url = "http://s3.amazonaws.com/verta-starter/census-train.csv"
train_data_filename = wget.detect_filename(train_data_url)
if not os.path.isfile(train_data_filename):
    wget.download(train_data_url)

test_data_url = "http://s3.amazonaws.com/verta-starter/census-test.csv"
test_data_filename = wget.detect_filename(test_data_url)
if not os.path.isfile(test_data_filename):
    wget.download(test_data_url)

In [ ]:
df_train = pd.read_csv(train_data_filename)
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv(test_data_filename)
X_test = df_test.iloc[:,:-1]
y_test = df_test.iloc[:, -1]


df_train.head()

## 1.2 Fit Model

In [ ]:
model = linear_model.LogisticRegression(max_iter=10**3)
model.fit(X_train, y_train)

model.score(X_train, y_train)

## 2. Register model for deployment

In [ ]:
registered_model = client.get_or_create_registered_model(name="census")

## 2.1 Create model version

In [ ]:
from verta.environment import Python

model_version = registered_model.create_standard_model_from_sklearn(
    model,
    environment=Python(requirements=["scikit-learn"]),
)

## 2.2 Profile training data

Here, Verta uploads profiles of your training data to faciliate downstream monitoring. Individual data points are *not* uploaded—the client only passes along numerical and categorical distributions of the columns in your data.

This step may take a few moments to run, depending on the dimensionality of your training data.

In [ ]:
model_version.log_training_data_profile(X_train, y_train)

These training data distributions can be viewed in the web app.

In [ ]:
model_version

## 3. Deploy model

In [ ]:
endpoint = client.create_endpoint(model_version.name.replace(" ", "-"))
endpoint.update(model_version, wait=True)

### 3.1 Make predictions

In [ ]:
deployed_model = endpoint.get_deployed_model()
for i, row in X_test.iterrows():
    print(deployed_model.predict([row.tolist()]))
    
    if i == 5:
        break

### 3.2 Fire alerts

---